# Requête HTTP 

Un requête HTTP est une requête basée sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisés dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas, nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme son nom l'indique de récupérer des informations en fonction de certains paramètres. 
- Le POST nécessite un envoi de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [49]:
import requests

In [50]:
url = "https://www.esiee.fr/"
response = requests.get(url)
response.status_code

200

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [51]:
type(response.content)

bytes

In [52]:
type(response.text)

str

Pour récupérer les 1000 premiers charactères de la page :

In [53]:
response.text[0:1000]

'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2025 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le classement de l&#039;Etudiant. Habilitée par la Commission des Titres d&#039;Ingénieur (CTI). Membre de la Conférence des Grandes Ecoles (CGE). " />\n<meta name="viewport" conte

Pour récupérer les headers HTTP de la réponse :

In [54]:
response.headers

{'Date': 'Mon, 20 Jan 2025 10:02:14 GMT', 'Server': 'Apache', 'Content-Language': 'fr', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=edge', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '15842', 'Content-Type': 'text/html; charset=utf-8', 'X-Varnish': '45580942 45023610', 'Age': '49', 'Via': '1.1 varnish (Varnish/7.1)', 'Accept-Ranges': 'bytes', 'Connection': 'keep-alive'}

On peut modifier les paramètres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent (identifiant de l'initiateur de la requête) et un timeout de 10 secondes :

In [55]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

b'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2025 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;\xc3\xa9cole de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande \xc3\xa9cole d&#039;ing\xc3\xa9nieur dans les domaines des transitions num\xc3\xa9rique, \xc3\xa9nerg\xc3\xa9tique et environnementale. Class\xc3\xa9e dans le groupe A, parmi les meilleures \xc3\xa9coles d&#039;ing\xc3\xa9nieur selon le classement de l&#039;Etudiant. Habilit\xc3\xa9e par la Commission des Titres d&#039;Ing\xc3\xa9nieur (CTI). Membr

## Exercice

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.

## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
- Récupérer le domaine en fonction d'un url

In [56]:
import requests
from requests.exceptions import RequestException, Timeout, HTTPError
import re
from urllib.parse import urlparse
import time


class HttpRequester:
    def __init__(self, user_agent: str):
        self.user_agent = user_agent
        self.headers = {'User-Agent': self.user_agent}

    def make_request(self, url: str, timeout: int = 10, retries: int = 3):
        attempt = 0
        while attempt < retries:
            try:
                print(f"Tentative {attempt + 1} pour récupérer {url}")
                response = requests.get(url, headers=self.headers, timeout=timeout)
                response.raise_for_status()
                return response.text
            except (RequestException, Timeout, HTTPError) as e:
                print(f"Erreur sur l'url : {url}: {e}")
                attempt += 1
                if attempt < retries:
                    print(f"re-essai dans 5 secondes...")
                    time.sleep(5)
                else:
                    raise e


def remove_superfluous_spaces(text: str) -> str:
    return ' '.join(text.split())


def clean_html_string(html: str) -> str:
    clean_text = re.sub(r'<[^>]+>', '', html)
    clean_text = re.sub(r'&amp;', '&', clean_text)
    clean_text = re.sub(r'&lt;', '<', clean_text)
    clean_text = re.sub(r'&gt;', '>', clean_text)
    clean_text = re.sub(r'&quot;', '"', clean_text)
    clean_text = re.sub(r'&apos;', "'", clean_text)
    return clean_text


def extract_domain(url: str) -> str:
    parsed_url = urlparse(url)
    return parsed_url.netloc



# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, il est normalement déjà installé, le cas échéant executez les lignes suivantes : 

In [57]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [58]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

Pour trouver tous les liens d'une page, on récupère la balise `a` qui permet de gérer les liens en HTML :

In [59]:
soup.find_all("a")[0:10]

[<a href="/#content">Aller au contenu</a>,
 <a href="/#menu">Aller au menu</a>,
 <a href="/plan-du-site/">Plan du site</a>,
 <a href="/actualites/journees-portes-ouvertes-2024-2025" target="_blank" title="Ouvre une nouvelle fenêtre">Bien choisir son école, c'est aussi la rencontrer ! Rendez-vous à ESIEE Paris le 8 février de 9h30 à 17h30 pour notre journée portes ouvertes.</a>,
 <a href="/"><img alt="ESIEE PARIS" class="a42-ac-replace-img" src="/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg"/></a>,
 <a href="/brochures-1">Brochures</a>,
 <a href="/informations/etudiantes-et-etudiants">Espace élèves</a>,
 <a href="/" hreflang="fr-FR" title="Français">
 <span>Fr</span>
 </a>,
 <a href="/en/" hreflang="en-US" title="English">
 <span>En</span>
 </a>,
 <a href="/candidater-1">Candidater</a>]

On peut aussi préciser la classe HTML qu'on veut récupérer :

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [60]:
soup.find_all(class_="toggler")[0:5]

[<button aria-controls="searchbox-header-form" aria-expanded="false" class="toggler">
 <i class="fa-solid fa-magnifying-glass"></i>
 <i class="fa-solid fa-xmark"></i>
 <span class="sr-only">
 <span class="display">Afficher</span><span class="hide">Masquer</span> la recherche
 		</span>
 </button>,
 <button aria-controls="submenu-40" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>L'école</button>,
 <button aria-controls="submenu-563" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Gouvernance et conseils</button>,
 <button aria-controls="submenu-65" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Départements d'enseignements et de recherche</button>,
 <button aria-controls="su

Pour récupérer le text sans les balises HTML :

In [61]:
soup.text[0:1000]

"\n\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nBien choisir son école, c'est aussi la rencontrer ! Rendez-vous à ESIEE Paris le 8 février de 9h30 à 17h30 pour notre journée portes ouvertes.\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsConseil scientifiqueAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environn

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

In [72]:
import random
from requests.exceptions import RequestException
from urllib.parse import urlparse, urljoin


class HttpRequester:
    def __init__(self, user_agents: list):
        self.user_agents = user_agents

    def _get_random_user_agent(self):
        return random.choice(self.user_agents)

    def make_request(self, url: str, timeout: int = 10, retries: int = 3):
        headers = {'User-Agent': self._get_random_user_agent()}
        for _ in range(retries):
            try:
                response = requests.get(url, headers=headers, timeout=timeout)
                response.raise_for_status()
                return response.text
            except RequestException:
                continue
        raise RequestException(f"Failed to retrieve {url} after {retries} attempts.")

    def get_soup(self, url: str):
        return BeautifulSoup(self.make_request(url), 'html.parser')

    def get_title(self, soup: BeautifulSoup):
        return soup.title.text if soup.title else 'No title found'

    def get_h1_list(self, soup: BeautifulSoup):
        return [h1.get_text(strip=True) for h1 in soup.find_all('h1')]

    def get_img_links(self, soup: BeautifulSoup, url: str):
        return [urljoin(url, img['src']) for img in soup.find_all('img') if img.get('src')]

    def get_main_txt(self, soup: BeautifulSoup):
        main = soup.find('main')
        return main.get_text(strip=True) if main else 'No main text found'

    def get_links(self, soup: BeautifulSoup, url: str):
        return list({urlparse(urljoin(url, a['href'])).netloc for a in soup.find_all('a') if a.get('href')})

    
def remove_superfluous_spaces(text: str) -> str:
    return ' '.join(text.split())

# User agents
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0"
]

url = "https://fr.wikipedia.org/wiki/Breton_(cheval)"
http_requester = HttpRequester(user_agents)

# Récupérer la page html
soup = http_requester.get_soup(url)

# Récupére les infos
title = http_requester.get_title(soup)
h1_list = http_requester.get_h1_list(soup)
img_list = http_requester.get_img_links(soup, url)
link_list = http_requester.get_links(soup, url)
main_text = http_requester.get_main_txt(soup)

# Affichage 
print(f"Title: {title}")
print(f"H1: {h1_list}")
print(f"Images: {img_list}")
print(f"Links: {link_list}")
print(f"Main Text: {remove_superfluous_spaces(main_text)}")


Title: Breton (cheval) — Wikipédia
H1: ['Breton (cheval)']
Images: ['https://fr.wikipedia.org/static/images/icons/wikipedia.png', 'https://fr.wikipedia.org/static/images/mobile/copyright/wikipedia-wordmark-fr.svg', 'https://fr.wikipedia.org/static/images/mobile/copyright/wikipedia-tagline-fr.svg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Logo_disambig.svg/20px-Logo_disambig.svg.png', 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Confusion_colour.svg/20px-Confusion_colour.svg.png', 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/83/Article_de_qualit%C3%A9.svg/15px-Article_de_qualit%C3%A9.svg.png', 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/Theme_de_qualite.svg/15px-Theme_de_qualite.svg.png', 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Horseicon.svg/40px-Horseicon.svg.png', 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Reine_de_Since.jpg/280px-Reine_de_Since.jpg', 'https://upload.wikimedia.org/wikipedia/co

# Exploitation des appels d'API



Losque le front du site récupère des données sur une API gérée par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL de la requête et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certains paramètres. L'API vérifie que dans les headers de la requête HTTP ces paramètres sont présents :
* un token généré à la volée avec des protocoles OAuth2 (ou moins développés).
* un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Google. 

Tips : 

- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes pour voir à quoi ressemble un appel API Google
- Utilisez beautiful soup pour convertir le contenu de la request en objet et accéder aux balises

In [74]:

url = "https://www.google.com/search?q=big+data"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}


response = requests.get(url, headers=headers)


if response.status_code == 200:
    print("Requête réussie !")
    
    # Parser le contenu de la page avec BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Trouver tous les titres des résultats de recherche (souvent dans des <h3>)
    results = soup.find_all('h3')


    for i, result in enumerate(results, 1):
        print(f"{i}. {result.get_text()}")

Requête réussie !


# Exercice Final  

Exercice Final
Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :
- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif) : les images

Tips : 

- Taper le nom de votre média favoris + RSS (par exemple : https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page 
- Trouver les catégories et les liens vers les articles

In [81]:
import xml.etree.ElementTree as ET

def extract_article_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        article_text = ' '.join([para.get_text() for para in paragraphs])
        page_title = soup.title.string if soup.title else 'No Title'
        return article_text, page_title
    else:
        return None, None

def get_news_from_rss(rss_url):
    response = requests.get(rss_url)
    if response.status_code != 200:
        print("Erreur lors de la récupération du flux RSS")
        return []
    
    root = ET.fromstring(response.content)
    articles = []
    
    for item in root.findall(".//item"):
        article = {}
        article['url'] = item.find('link').text
        parsed_url = urlparse(article['url'])
        article['domain'] = parsed_url.netloc
        article['title'] = item.find('title').text
        category_element = item.find('category')
        article['category'] = category_element.text if category_element is not None else 'No Category'
        article_text, page_title = extract_article_content(article['url'])
        article['article_text'] = article_text
        article['page_title'] = page_title
        articles.append(article)
    
    return articles

rss_url = 'https://www.lemonde.fr/rss/en_continu.xml'
articles = get_news_from_rss(rss_url)
for article in articles:
    print("Titre :", article['title'])
    print("Categorie :", article['category'])
    print("URL :", article['url'])
    print("Domaine :", article['domain'])
    print("Texte :", article['article_text'][:200])
    print("Titre de la page :", article['page_title'])


Titre : Open d’Australie : Jannik Sinner qualifié, Iga Swiatek épargnée par l’AMA
Categorie : No Category
URL : https://www.lemonde.fr/sport/article/2025/01/20/open-d-australie-l-italien-jannik-sinner-tenant-du-titre-se-qualifie-pour-les-quarts-de-finale_6506596_3242.html
Domaine : www.lemonde.fr
Texte : Consulterle journal BYD Sealion, la voiture de Big Brother « Le pouvoir des géants de la tech érode la démocratie » Crise politique, consommation en berne, hausse du chômage : « La Corée du Sud a plus
Titre de la page : Open d’Australie : Jannik Sinner qualifié, comme Iga Swiatek, épargnée par l’Agence mondiale antidopage
Titre : Comment le gouvernement Bayrou compte économiser des milliards d’euros grâce à la loi spéciale
Categorie : No Category
URL : https://www.lemonde.fr/politique/article/2025/01/20/finances-publiques-comment-l-etat-compte-economiser-des-milliards-d-euros-grace-a-la-loi-speciale_6506385_823448.html
Domaine : www.lemonde.fr
Texte : Consulterle journal BYD Sealion, l